# Getting started with graph convolutions

**Author**: Dennis Ramondt

**Conference**: PyData Eindhoven 2020

**Talk**: Monitoring a TV streaming service with AI - from PageRank to graph convolutions

This notebook gives an primer to using graph convolutions, simple node embeddings and supervised pagerank. We use the well known public `Cora` dataset.

All you'll need is `tensorflow 2.1.0`, as it's listed in the requirements.txt.

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import sys

## 0. Data preparation

The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

In [ ]:
from gcn.utils import *

In [ ]:
DATASET = 'cora'
X, A, y = load_data(path='gcn/data/cora/', dataset=DATASET)

In [ ]:
num_nodes = A.shape[1]
num_features = X.shape[1]
num_categories = y.shape[1]

We use a simple utility function to create training masks. These can be used during training to effectively hold out some part of the training data for validation afterwards.

In [ ]:
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In order to do proper graph deep learning, it's important to normalize the feature and adjacency matrices.

In [ ]:
X /= X.sum(1).reshape(-1, 1)

In [ ]:
A_ = preprocess_adj(A, symmetric=True)

## 1. Basic Graph Convolution

The basic graph convolution, as outlined in the talk, works by propagating node feature information using the adjacency matrix. This implementation is inspired by Thomas Kipf's seminal paper: https://arxiv.org/pdf/1609.02907.pdf

In [ ]:
from tensorflow.keras.layers import Dense, Input, Add, Dropout
from gcn.layers.graph import PersonalizedPageRank, GraphConvolution
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam

In order to get the graph convolutions working, we need to pass both the feature matrix and the adjacency matrix as Layer inputs:

In [ ]:
G = Input(shape=(num_nodes, num_nodes))
X_in = Input(shape=(num_nodes, num_features,))
graph = [np.expand_dims(X, 0), np.expand_dims(A_.todense(), 0)]

Convolutional layers can be stacked at will, depending on how far you want the messages to be passed through the graph.

In [ ]:
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, activation='relu')([H,G])
H = Dropout(0.5)(H)
H = GraphConvolution(32, activation='relu')([H,G])
H = Dropout(0.5)(H)
H = GraphConvolution(32, activation='relu')([H,G])
Y = Dense(num_categories, activation='softmax')(H)

model = Model(inputs=[X_in,G], outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), sample_weight_mode='temporal')

In [ ]:
model.fit(graph, np.expand_dims(y_train, 0), sample_weight=np.expand_dims(train_mask, 0),
          batch_size=num_nodes, epochs=50, shuffle=False, verbose=2)

In [ ]:
preds = model.predict(graph, batch_size=A.shape[0])

For evaluating the predictions, we use the indices for the training mask to split into validation and training sets to see the accuracy and categorical cross entropy loss.

In [ ]:
train_val_loss, train_val_acc = evaluate_preds(preds[0], [y_train, y_val],
                                               [idx_train, idx_val])

In [ ]:
train_val_acc

As we can see, the validation accuracy is around 75%. Optimal benchmarks for `Cora` are somewhere in the 80s.

## 2. Personalized PageRank

An alternative approach is to use the pagerank algorithm to propagate information through the graph. It also uses the adjacency matrix, but can pass messages deeper in the graph due to the multiple (here 10) power iterations.

Note that it doesn't use a kernel to weight the message passing so will not learn which adjacency weights to prefer during propagation. This means the algorithm is not suitable when the adjacency matrix is very sparse and the node features themselves contain little signal.

Furthermore, a nice feature is that it is used as a final layer, hence you can use normal dense layers to learn information from the features themselves before propagating the learned representation.

In [ ]:
H = Dense(64)(X_in)
H = Dense(64)(H)
H = Dense(num_categories, activation='softmax')(H)
Y = PersonalizedPageRank(alpha=0.1, niter=10, keep_prob=0.5)([H,G])

model = Model(inputs=[X_in,G], outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), sample_weight_mode='temporal')

In [ ]:
model.fit(graph, np.expand_dims(y_train, 0), sample_weight=np.expand_dims(train_mask, 0),
          batch_size=num_nodes, epochs=50, shuffle=False, verbose=2)

In [ ]:
preds = model.predict(graph, batch_size=A.shape[0])

In [ ]:
train_val_loss, train_val_acc = evaluate_preds(preds[0], [y_train, y_val],
                                               [idx_train, idx_val])

In [ ]:
train_val_acc

Interestingly, we get much higher accuracy here, suggesting that the node features already contain a lot of information and pagerank helps to send this deep enough into the graph.

## 3. Node feature embedding

In many real-world scenarios not every node in the graph will be similar. For example, different node types may have different feature sets. This heterogeneity means graph convolutions will not work out of the box. A simple approach to fix this is to first embed the different features into the same dimension and then add them together. After that, simple graph convolutions can be used.

Here, we have slices the node feature matrix into two distinct feature sets. We have then given one half of the nodes the first feature set, and the other half the second feature set. Let's see if they are still able to learn anything meaningful.

In [ ]:
indices = np.random.choice([0,1], p=[0.5, 0.5], size=num_features)
Xs = [X[:,np.where(indices==0)[0]], X[:,np.where(indices==1)[0]]]
G = Input(shape=(num_nodes, num_nodes))
Xs_in = [Input(shape=(X.shape[0], X.shape[1])) for X in Xs]

graphs = [np.expand_dims(X, 0) for X in Xs] + [np.expand_dims(A_.todense(), 0)]

In [ ]:
Xs_embed = [Dense(24, use_bias=False)(X) for X in Xs_in]
H = Add()(Xs_embed)
H = Dropout(0.5)(H)
H = GraphConvolution(32, activation='relu')([H,G])
H = Dropout(0.5)(H)
H = GraphConvolution(32, activation='relu')([H,G])
H = Dropout(0.5)(H)
H = GraphConvolution(32, activation='relu')([H,G])
Y = Dense(num_categories, activation='softmax')(H)

model = Model(inputs=[Xs_in,G], outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), sample_weight_mode='temporal')

In [ ]:
model.fit(graphs, np.expand_dims(y_train, 0), sample_weight=np.expand_dims(train_mask, 0),
          batch_size=num_nodes, epochs=50, shuffle=False, verbose=2)

In [ ]:
preds = model.predict(graphs, batch_size=num_nodes)

In [ ]:
train_val_loss, train_val_acc = evaluate_preds(preds[0], [y_train, y_val],
                                               [idx_train, idx_val])

In [ ]:
train_val_acc

The result is a bit lower than using graph convolutions on the full set of features for all nodes, but that is to be expected given that we have essentially removed half of the meaningful information when simulating the split into two disjoint node types.

## 4. Recurrent GCNs

Graph Convolutions can also be used in combination with time series. The PEMS-08 traffic dataset consists of 5-minute `flow`, `occupy` and `speed` measurements (aggregated from 30 second raw data) of 170 sensors along a network of highways in the major metropolitan areas in California. The objective is to predict sensor values at some future timestep `T`, making use of the road network structure. In this implementation we choose to predict one of the input features `T` timesteps ahead.

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.metrics import *

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.layers import Reshape, GRU, Lambda, TimeDistributed
import tensorflow.keras.backend as K
from gcn.layers.graph import GraphConvolution
from gcn.utils import DataGenerator

We read the feature matrix, normalize it, and use the same adjacency matrix normalization as before.

In [ ]:
# node features
X = np.load('gcn/data/pems/pems08.npz')['data']
X /= X.max(axis=0, keepdims=True)
X = np.expand_dims(X, 0)

# adjacency matrix
distance = pd.read_csv('gcn/data/pems/distance.csv').drop_duplicates()
A = sp.coo_matrix((distance['cost'], (distance['from'], distance['to'])), shape=(X.shape[2], X.shape[2]))
A_ = np.expand_dims(preprocess_adj(A, symmetric=False).todense(), 0).astype('float32')

We define a sequence length of 1 day:

In [ ]:
seq_len = 288
n_nodes = A.shape[1]
n_features = X.shape[3]
train_frac = 0.95
train_idx = int(X.shape[1] * train_frac)

In [ ]:
X_in = Input(shape=(seq_len, n_nodes, n_features,))
G = Input(shape=(n_nodes, n_nodes))

This data generator class returns node feature tensors of length `seq_len` together with the adjacency matrix for training and testing.

In [ ]:
train_gen = DataGenerator(A_, X[:,:train_idx], target_distance=1, seq_length=seq_len, batch_size=64)
test_gen = DataGenerator(A_, X[:,train_idx:], target_distance=1, seq_length=seq_len, batch_size=64)

### 4.1 Approach 1: flattening sequences

The first and most naieve approach is to flatten the node feature sequences in the first layer into a single feature vector. That way they can be directly be passed through convolutional layers. Although the approach trains fast, there are some methodological concerns as temporal information is essentially jumbled when learning the convolutional weights.

In [ ]:
H = Reshape((n_nodes, seq_len * n_features))(X_in)
H = GraphConvolution(32 * n_features, activation='relu')([H, G])
H = Dropout(0.2)(H)
H = GraphConvolution(16 * n_features, activation='relu')([H, G])
H = Dropout(0.2)(H)
H = Reshape((-1, n_nodes * n_features))(H)
H = GRU(32, return_sequences=True)(H)
H = Dropout(0.2)(H)
H = GRU(64)(H)
H = Dropout(0.2)(H)
Y = Dense(n_nodes, activation='sigmoid')(H)

In [ ]:
model = Model(inputs=[X_in, G], outputs=Y)
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

In [ ]:
model.fit(train_gen, epochs=5, validation_data=test_gen)

Validating the quality of predictions

In [ ]:
outputs = [t[1] for t in test_gen]
y_true = np.concatenate(outputs, axis=0)
y_pred = model.predict(test_gen)

In [ ]:
print(f'Mean Squared Error {mean_squared_error(y_true, y_pred)}')
print(f'Mean Absolute Error {mean_absolute_error(y_true, y_pred)}')

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(y_true[:,1])
plt.plot(y_pred[:,1])

### 4.2 Approach 2: convolutions at each timestep
A more robust approach is to apply a graph convolution at each timestep, before passing the output to a conventional recurrent layer. This approach is inspired by a recent paper from Zhao et al. (2018). They actually build a convolutional layer into the GRU Cell entry gate itself, but the current Keras implementation for recurrent cells is so entangled that it is a chore to replicate and insert the convolutions.

Instead, one can apply the graph convolutions in time distributed fashion with Keras, and then pass the result into a conventional recurrent layer. Unfortunately (as is the case with the paper's implementation as well), a full convolution that includes the weight kernel is extremely slow. I therefore chose to only implement the propagation step to reduce the amount of weights that need to be trained.

**Reference**
Zhao et al. (T-GCN: A Temporal Graph Convolutional Network for Traffic Prediction (IEEE Transactions on Intelligent Transportation Systems 2019))

https://arxiv.org/abs/1811.05320

In [ ]:
H = TimeDistributed(Lambda(lambda x: K.batch_dot(A_, x)))(X_in)
H = TimeDistributed(Lambda(lambda x: K.batch_dot(A_, x)))(H)
H = Reshape((seq_len, n_nodes * n_features))(H)
H = GRU(32, return_sequences=True)(H)
H = Dropout(0.2)(H)
H = GRU(64)(H)
H = Dropout(0.2)(H)
Y = Dense(n_nodes, activation='sigmoid')(H)

In [ ]:
model = Model(inputs=[X_in, G], outputs=Y)
model.compile(optimizer="adam", loss="mae", metrics=["mse"])

In [ ]:
model.fit(train_gen, epochs=5, validation_data=test_gen)

Validating the quality of predictions

In [ ]:
outputs = [t[1] for t in test_gen]
y_true = np.concatenate(outputs, axis=0)
y_pred = model.predict(test_gen)

In [ ]:
print(f'Mean Squared Error {mean_squared_error(y_true, y_pred)}')
print(f'Mean Absolute Error {mean_absolute_error(y_true, y_pred)}')

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(y_true[:,1])
plt.plot(y_pred[:,1])